<a href="https://colab.research.google.com/github/dsk-003/Air-Quality/blob/main/Copy_of_Custom_NER_17_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy-transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U spacy
import spacy


In [ ]:

spacy.__version__

'3.7.2'

In [ ]:
!nvidia-smi

Thu Jan 18 08:59:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import json
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
import json
cv_data=json.load(open('/content/drive/MyDrive/cv_data/output/training_data.json','r'))

In [ ]:
cv_data[0]

['SYSTEMS ENGINEER NEHA RANI Phone: +91-8340674163 Email:nehdstar@gmail.com LinkedIn:https://www.linkedin.com/in/neha-rani- a9839b110/ Location: Pune, MAHARASHTRA-411057 Objective 2 years experienced skilled Engineer with proven success in building successful algorithms & predictive models . Highly adept at clustering & classification(ML), Natural Language Processing, computer vision and Deep Learning to increase business efficiency. Technical Skills Tools: Microsoft Azure, Labellmg Technology: Machine learning, Natural language processing, DEEP Learning, Computer vision and Neural Networks.Languages: Python Work Experience SYSTEMS ENGINEER INFOSYS LIMITED, PUNE  March 2019 PRESENT Project : Sentiment Analysis Project Description : Analysis and Classification of sentiment of customers experience in text form. It involved identification, extraction and scoring of consumer feelings and opinions as it appears in social media, customer surveys, emails, blogs etc. it predicted whether the c

In [ ]:
len(cv_data)

10

In [ ]:
type(cv_data)

list

In [ ]:
!pip install spacy-transformers

import spacy_transformers

In [ ]:
from transformers import AutoTokenizer, RobertaModel
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/cv_data/output/config.cfg config.cfg

2024-01-18 06:10:03.167976: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-18 06:10:03.168030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-18 06:10:03.169626: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-18 06:10:04.676993: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
def get_spacy_doc(file,data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text,annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity==True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')

      except:
        continue

      if span is None:
        err_data = str([start, end]) + "     " + str(text)  + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [ ]:
from sklearn.model_selection import train_test_split
import spacy
train,test = train_test_split(cv_data, test_size=0.2)

In [ ]:
len(train),len(test)

(8, 2)

In [ ]:
file = open('error.txt','w')

db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')


db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 2/2 [00:00<00:00, 11.30it/s]


In [ ]:
#!python -m spacy debug config config.cfg
!python -m spacy train /content/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2024-01-18 06:12:10.372273: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-18 06:12:10.372337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-18 06:12:10.373691: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-18 06:12:11.421394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint

### Testing

In [ ]:
nlp = spacy.load('/content/output/model-best')

NameError: name 'spacy' is not defined

In [ ]:
doc = nlp('Aniruddha Kawarase anujabagad18@gmail.com +91-8421675729 Profile Summary  2 year Experience Machine Learning Engineer with over 2 years of experience in IT Industry.')

for ent in doc.ents:
  print(ent.text, "->>>>",ent.label_)

Aniruddha Kawarase ->>>> Name of candidate


In [ ]:
!pip install PyMuPDF


In [ ]:
import sys, fitz

In [ ]:
fname = '/content/drive/MyDrive/cv_data/Neha.pdf'

In [ ]:
doc = fitz.open(fname)
doc


Document('/content/drive/MyDrive/cv_data/Neha.pdf')

In [ ]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [ ]:
text

' SYSTEMS ENGINEER \nNEHA RANI \nPhone: +91-8340674163 \nEmail:nehdstar@gmail.com \nLinkedIn:https://www.linke \ndin.com/in/neha-rani- \na9839b110/ \nLocation: Pune, \nMAHARASHTRA-411057 \n \n \nObjective \n2 years experienced skilled Engineer with proven success in building successful algorithms & \npredictive models . Highly adept at clustering & classification(ML), Natural Language \nProcessing, computer vision and Deep Learning to increase business efficiency. \nTechnical Skills \n \nTools: Microsoft Azure, Labellmg \nTechnology: Machine learning, Natural language processing, DEEP \nLearning, Computer vision and Neural Networks. \nLanguages: Python \n \nWork Experience \n \nSYSTEMS ENGINEER \nINFOSYS LIMITED, PUNE • March 2019 – PRESENT \n \n• Project : Sentiment Analysis \nProject Description : Analysis and Classification of sentiment of \ncustomers experience in text form. It involved identification, \nextraction and scoring of consumer feelings and opinions as it \nappears in so

In [ ]:
text = text.strip()

In [ ]:
text = ' '.join(text.split())

In [ ]:
doc = nlp(text)

for ent in doc.ents:
  print(ent.text, "->>>>",ent.label_)